In [1]:
import os
import csv
import pandas as pd
import numpy as np
import itertools

In [2]:
file = r'C:\DTU\Data\201901_JanuaryExpt\bh2\2\2019-02-27-13-19-57_1\tobiiGazeLog-2019-2-27-13-19-57.csv'


fGazeLog = open(file, encoding='utf-8', newline='')
readerGazeLog = csv.reader(fGazeLog, quotechar=None)
gazeLog = list(readerGazeLog)
gazeLog.remove(gazeLog[0])
gazeLog.remove(gazeLog[-1])


# first create a list of times in gaze log
timeStrGazeLog = [item3[0] for item3 in gazeLog]





In [3]:
def gazeConvert2ColumnsTo1(GazeLog, columnIndwValidity_list, indValidity):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    #columnInd_list = [joinColumn1_1, joinColumn1_2, joinColumn2_1, joinColumn2_2]
    
    # number of columns in the final dictionary
    nColumns = int(len(columnIndwValidity_list)/2)
    
    # dictionary of columns that are to be joined later
    columns_beforeDecimal = dict()
    columns_afterDecimal = dict()
    
    # dictionary of joined columns
    columnsFinal = dict()
    
    # dictionary to find and equalize missing values in every column
    missingVal_column = dict()
    missingVal = list()
    
    # find correct index of validity column to be used, to find the actual columns relative to that
    columnsValidity_inUse = list()
    
    for ind, row in enumerate(GazeLog):
        #print(ind)
        #print(sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])

        columnsValidity = (sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])
        columnsValidity_inUse.append(int(columnsValidity))
    
    columnsValidity_inUse = np.array(columnsValidity_inUse)
    
    columnInd_list = [[columnsValidity_inUse+i] for i in columnIndwValidity_list]
    
    for ind in range(0, nColumns):
        
        dict_name = 'column' + str(ind+1)
        columnsFinal[dict_name] = list()
        columns_afterDecimal[dict_name] = list()
                
        #for indItem, item4 in enumerate(GazeLog):
        #    if 'Invalid' not in item4:
        #        if columnInd_list[2*ind+1][0][indItem] < len(item4):
        #            columns_afterDecimal[dict_name].append(item4[columnInd_list[2*ind+1][0][indItem]])
        #        else:
        #            columns_afterDecimal[dict_name].append('0')
        #    else:
        #        columns_afterDecimal[dict_name].append('nan')
        
                
        columns_beforeDecimal[dict_name] = [item4[columnInd_list[2*ind][0][indItem]] if 'Invalid' not in item4 else 'nan' for indItem, item4 in enumerate(GazeLog)]
        columns_afterDecimal[dict_name] = [item4[columnInd_list[2*ind+1][0][indItem]] if 'Invalid' not in item4 and columnInd_list[2*ind+1][0][indItem] < len(item4) else 'nan' for indItem, item4 in enumerate(GazeLog)]
        
        
        for i in range(0, len(columns_beforeDecimal[dict_name])):
            if 'Valid' not in columns_beforeDecimal[dict_name][i] and 'Valid' not in columns_afterDecimal[dict_name][i]:
                if 'nan' not in columns_beforeDecimal[dict_name][i] and 'nan' not in columns_afterDecimal[dict_name][i]:
                    if float(columns_afterDecimal[dict_name][i]) > 0: 
                        columnsFinal[dict_name].append(float(columns_beforeDecimal[dict_name][i]+'.'+columns_afterDecimal[dict_name][i]))
                    else:
                        columnsFinal[dict_name].append(np.nan)
                else:
                    columnsFinal[dict_name].append(np.nan)
            else:
                # Rarely, the pupil size is a whole number
                columnsFinal[dict_name].append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
                # right or left eye has whole number pupil size
    
        missingVal_column[dict_name] = np.argwhere(np.isnan(columnsFinal[dict_name]))
        missingVal_column[dict_name] = list(itertools.chain.from_iterable(missingVal_column[dict_name])) # flatten the list
        
        missingVal.extend(missingVal_column[dict_name])
        
        
    
    missingVal = sorted(set(missingVal))
    
    
    # if one of the columns are nan, the other one is converted too
    for column in range(0, nColumns):
        dict_name = 'column' + str(column+1)
        for ind in missingVal:
            if ind < len(columnsFinal[dict_name]):
                columnsFinal[dict_name][ind] = np.nan
                
    
    
    
    #print(len(columnsFinal['column1']), len(columnsFinal['column2']))
    
    
    

    return columnsFinal

In [5]:
print('gazeOrigin')
gazeOriginUCS_validityL = 0
gazeOriginUCS_indWrtValidityL = [1, 2, 3, 4, 5, 6]
gazeOriginUCS_Left = gazeConvert2ColumnsTo1(gazeLog, gazeOriginUCS_indWrtValidityL, gazeOriginUCS_validityL)
    
gazeOriginUCS_validityR = 1
gazeOriginUCS_indWrtValidityR = [1, 2, 3, 4, 5, 6]
gazeOriginUCS_Right = gazeConvert2ColumnsTo1(gazeLog, gazeOriginUCS_indWrtValidityR, gazeOriginUCS_validityR)
    
# find the average of each to compute the gaze origin
gazeOriginUCS = dict()
gazeOriginUCS['x'] = np.array([(v+gazeOriginUCS_Right['column1'][i])/2 for i, v in enumerate(gazeOriginUCS_Left['column1'])])
gazeOriginUCS['y'] = np.array([(v+gazeOriginUCS_Right['column2'][i])/2 for i, v in enumerate(gazeOriginUCS_Left['column2'])])
gazeOriginUCS['z'] = np.array([(v+gazeOriginUCS_Right['column3'][i])/2 for i, v in enumerate(gazeOriginUCS_Left['column3'])])
    
    
# Location of eye gaze on screen in the UCS by Tobii (in mm)
print('gazePoint')
gazePointUCS_validityL = 2
gazePointUCS_indWrtValidityL = [1, 2, 3, 4, 5, 6]
gazePointUCS_Left = gazeConvert2ColumnsTo1(gazeLog, gazePointUCS_indWrtValidityL, gazePointUCS_validityL)
    
gazePointUCS_validityR = 3
gazePointUCS_indWrtValidityR = [1, 2, 3, 4, 5, 6]
gazePointUCS_Right = gazeConvert2ColumnsTo1(gazeLog, gazePointUCS_indWrtValidityR, gazePointUCS_validityR)
    
# find the average of gaze point in UCS
gazePointUCS = dict()
gazePointUCS['x'] = np.array([(v+gazePointUCS_Right['column1'][i])/2 for i, v in enumerate(gazePointUCS_Left['column1'])])
gazePointUCS['y'] = np.array([(v+gazePointUCS_Right['column2'][i])/2 for i, v in enumerate(gazePointUCS_Left['column2'])])
gazePointUCS['z'] = np.array([(v+gazePointUCS_Right['column3'][i])/2 for i, v in enumerate(gazePointUCS_Left['column3'])])
    
# find the distance between the screen and eyes, dividing by 10 to get cm from mm 
distanceEyeGaze = [(np.sqrt((gazePointUCS['x'][i]-gazeOriginUCS['x'][i])**2 + (gazePointUCS['y'][i]-gazeOriginUCS['y'][i])**2 + (gazePointUCS['z'][i]-gazeOriginUCS['z'][i])**2))/10 for i, v in enumerate(gazePointUCS['x'])]
    


# Location of eye gaze on screen in the ADCS by Tobii (in arbitrary units)
gazePointADCS_indWrtValidityL = [-4, -3, -2, -1]
gazePointADCS_validityL = 3
gazePointADCS_Left_au = gazeConvert2ColumnsTo1(gazeLog, gazePointADCS_indWrtValidityL, gazePointADCS_validityL)
    
gazePointADCS_indWrtValidityR = [-4, -3, -2, -1]
gazePointADCS_validityR = 4
gazePointADCS_Right_au = gazeConvert2ColumnsTo1(gazeLog, gazePointADCS_indWrtValidityR, gazePointADCS_validityR)
    
    
# gazePointADCS is in arbitrary units and needs to be converted to cm  
screenLength = 59
screenWidth = 34.5
    
gazePointADCS_Left_au['column1'] = [i*59 for i in gazePointADCS_Left_au['column1']]
gazePointADCS_Left_au['column2'] = [i*34.5 for i in gazePointADCS_Left_au['column2']]
    
gazePointADCS_Right_au['column1'] = [i*59 for i in gazePointADCS_Right_au['column1']]
gazePointADCS_Right_au['column2'] = [i*34.5 for i in gazePointADCS_Right_au['column2']]
    
x = [(v+gazePointADCS_Right_au['column1'][i])/2 for i, v in enumerate(gazePointADCS_Left_au['column1'])]
y = [(v+gazePointADCS_Right_au['column2'][i])/2 for i, v in enumerate(gazePointADCS_Left_au['column2'])]


x_deg = np.array([v*screenLength/distanceEyeGaze[i] for i, v in enumerate(x)])
y_deg = np.array([v*screenWidth/distanceEyeGaze[i] for i, v in enumerate(y)])
    

gazeOrigin
gazePoint


NameError: name 'gazePointADCS' is not defined

In [6]:
x_deg = np.array([v*screenLength/distanceEyeGaze[i] for i, v in enumerate(x)])
y_deg = np.array([v*screenWidth/distanceEyeGaze[i] for i, v in enumerate(y)])


In [7]:
df = pd.DataFrame({'x': x, 'y': y, 'time': timeStrGazeLog, 'xDeg': x_deg, 'yDeg': y_deg})

In [8]:
df.to_excel(r'C:\DTU\Data\201901_JanuaryExpt\eyeData.xlsx', header=False, index=False)